<a href="https://colab.research.google.com/github/geersenthil/Topic-Modeling-/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import required libraries

In [1]:
pip install pyLDAvis


     |████████████████████████████████| 1.7 MB 3.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=195d6d8d67b565fe3ab7721cfa2285f21ec313cacccba19375cb8af29630ea7b
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [110]:
import re
import pandas as pd
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing import strip_tags, strip_numeric, strip_multiple_whitespaces, stem_text, strip_punctuation, remove_stopwords
from gensim.parsing import preprocess_string



import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#Dataset
from sklearn.datasets import fetch_20newsgroups

#Spacy
import spacy
nlp = spacy.load('en_core_web_sm')

#Visuals
import pyLDAvis
import pyLDAvis.gensim_models

#warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Load Data into Dataframe

In [116]:
news_group = fetch_20newsgroups()

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

In [117]:
news_df = pd.DataFrame({'news': news_group_data})

In [118]:
news_data = news_df['news'].sample(500)

Pre-process data

Lemmatization to get base words

In [119]:
def lemmatization(texts, allowed_postags=["NOUN","ADJ","VERB","ADV"]):
  nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])
  texts_out = []
  for text in texts:
    doc = nlp(text)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
            new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return (texts_out)
 

In [120]:

lemmatizated_text =  lemmatization(news_data)


In [137]:
print(lemmatizated_text)


['subject so far so good line article armstrng@cs.dal.ca write article write may really dumb will ask anyways know can never live requirement right may wrong understanding still try seem spend life try reach goal can never achieve know save faith work mean save think tell work dead paraphrase how work so long think good thing rather come want miss point more love more come love love when find do good sign try even harder sign seek when aware presence usually want want strenth love empower weakness armstrong@husky1.stmarys.ca | att!clyde!watmath!water!dalcs!armstrng apologize moderator first quote delete would like respond goal can never achieve reward come trying make clear claim continue strain prize only live command stagnate become lukewarm spit say love obey comand obedience straining achieve of course require work part quote care believe matter result belief work truly faith believe will act faith lie believe first place first line very good point obedience obligation grudgery des

In [146]:
stop_words = stopwords.words('english')
print(len(stop_words))
stop_words.extend(['from', 'subject', 're', 'edu', 'use','cdw','would','line','article'])
print(len(stop_words))

179
188


In [147]:
#tokenize
def gen_words(texts):
  final = []

  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    for w in new:
      if w in stop_words:
        new.remove(w)
    final.append(new)
  return (final)

data_word = gen_words(lemmatizated_text)

In [148]:
print(data_word)

[['far', 'good', 'armstrng', 'cs', 'dal', 'ca', 'write', 'article', 'write', 'may', 'really', 'dumb', 'ask', 'anyways', 'know', 'never', 'live', 'requirement', 'right', 'may', 'wrong', 'understanding', 'still', 'try', 'seem', 'spend', 'life', 'try', 'reach', 'goal', 'never', 'achieve', 'know', 'save', 'faith', 'work', 'mean', 'save', 'think', 'tell', 'work', 'dead', 'paraphrase', 'work', 'long', 'think', 'good', 'thing', 'rather', 'come', 'want', 'miss', 'point', 'love', 'come', 'love', 'love', 'find', 'good', 'sign', 'try', 'even', 'harder', 'sign', 'seek', 'aware', 'presence', 'usually', 'want', 'want', 'strenth', 'love', 'empower', 'weakness', 'armstrong', 'husky', 'stmarys', 'ca', 'att', 'clyde', 'watmath', 'water', 'dalcs', 'armstrng', 'apologize', 'moderator', 'first', 'quote', 'delete', 'like', 'respond', 'goal', 'never', 'achieve', 'reward', 'come', 'trying', 'make', 'clear', 'claim', 'continue', 'strain', 'prize', 'live', 'command', 'stagnate', 'become', 'lukewarm', 'spit', 's

In [149]:
#id2word

id2word = corpora.Dictionary(data_word)
corpus = []
for text in data_word:
  new = id2word.doc2bow(text)
  corpus.append(new)

#id and frequency of word
print (corpus[0][0:20])

[(0, 1), (1, 3), (2, 1), (3, 2), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 3), (16, 2), (17, 1), (18, 1), (19, 1)]


In [150]:
word = id2word[[0][:1][0]]
print(word)

accomplish


In [151]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1], id2word = id2word, num_topics = 15, random_state = 100, update_every=1, chunksize=100, passes = 10, alpha="auto")

In [152]:
test_doc = corpus[-1]
vector = lda_model[test_doc]
print(vector)

# sort by match f
def Sort(sub_li):
  sub_li.sort(key = lambda x: x[1])
  sub_li.reverse()
  return(sub_li)
new_vector = Sort(vector)
print(new_vector)



[(0, 0.012399265), (2, 0.017157104), (3, 0.10073508), (4, 0.6643371), (5, 0.03431809), (6, 0.07461863), (8, 0.010864182), (9, 0.011543243), (10, 0.01830826), (13, 0.053726252)]
[(4, 0.6643371), (3, 0.10073508), (6, 0.07461863), (13, 0.053726252), (5, 0.03431809), (10, 0.01830826), (2, 0.017157104), (0, 0.012399265), (9, 0.011543243), (8, 0.010864182)]


Visualise Data

In [153]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.149153 -0.359580       1        1  34.383884
13     0.086754 -0.348781       2        1  13.437383
6     -0.354564  0.143827       3        1  10.094412
5      0.237989 -0.220514       4        1   6.680895
8      0.306055 -0.088458       5        1   5.456570
3      0.328555  0.120035       6        1   4.845377
10    -0.014592  0.363581       7        1   4.142944
12    -0.018254 -0.194501       8        1   4.026731
14    -0.335431 -0.057455       9        1   3.911791
2     -0.192189 -0.108343      10        1   3.686150
0     -0.175522  0.188967      11        1   2.782102
7     -0.019128  0.181455      12        1   2.163762
11     0.199358  0.280783      13        1   2.147045
1      0.008982  0.017744      14        1   1.297172
9      0.091140  0.081240      15        1   0.943781, topic_info=         Term        Freq       Total Category  logprob  loglift
739      year  143.000000  143.000000  Default  10.0000  10.0000
387       key  109.000000  109.000000  Default   9.0000   9.0000
246    people  194.000000  194.000000  Default   8.0000   8.0000
1073     team   74.000000   74.000000  Default   7.0000   7.0000
1197    child   79.000000   79.000000  Default   6.0000   6.0000
...       ...         ...         ...      ...      ...      ...
6362       er    2.197479    2.918089  Topic15  -5.3528   4.3794
6218  leavitt    1.771828    2.492432  Topic15  -5.5681   4.3218
185    window   14.483415   63.512906  Topic15  -3.4671   3.1848
167   manager    5.104766   16.007466  Topic15  -4.5100   3.5202
181   specify    3.229424    6.736526  Topic15  -4.9678   3.9278

[268 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
4501     10  0.906673        about
3206     14  0.767209  acknowledge
2170      8  0.970656     addition
6480      9  0.963414        adult
2318      3  0.975691          age
...     ...       ...          ...
148      15  0.002495        write
739       1  0.647505         year
739       2  0.097474         year
739       6  0.020887         year
739      10  0.229760         year

[499 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 14, 7, 6, 9, 4, 11, 13, 15, 3, 1, 8, 12, 2, 10])